In [1]:
from RiotRequest import RiotRequest
from RiotAPI import *
from RiotUtil import *

import pickle
import time
import datetime
import numpy as np

from IPython.display import clear_output

## Extract Unique Match ID  
how to concatenate dicionary: https://stackoverflow.com/questions/1781571/how-to-concatenate-two-dictionaries-to-create-a-new-one-in-python  
```python
result = dict(dic1, **dic2)
result.update(dic3)
```

In [2]:
######## LOAD AND SAVE AS PICKLE ########

def load_pickle(fn):
    with open('pickle/{}.pickle'.format(fn), 'rb') as f:
        file = pickle.load(f)
    return file

def save_pickle(data, fn):
    with open('pickle/{}.pickle'.format(fn), 'wb') as f:
        pickle.dump(data, f)

In [84]:
######### DIAMOND 1, 2 ###########

# load
dia1 = load_pickle("match_DIAMOND1")
dia2 = load_pickle("match_DIAMOND2")

# append
dia = dict(dia1, **dia2)

# full list of id
fullList = [match['gameId'] for matchList in dia.values() for match in matchList]

# extract unique list of id and save
unq_games = list(set(fullList))
save_pickle(unq_games,"unique_matchId_diamond")

len(unq_games) # 298만 6533개

2986533

In [3]:
########## MASTER, GRANDMASTER, CHALLENGER ############

master = load_pickle('match/match_MASTER')
grand = load_pickle('match/match_GRANDMASTER')
challenger = load_pickle('match/match_CHALLENGER')

masterId = [match['gameId'] for matchList in master.values() for match in matchList]
grandId = [match['gameId'] for matchList in grand.values() for match in matchList]
challengerId = [match['gameId'] for matchList in challenger.values() for match in matchList]

In [4]:
unq_game_upper = list(set(masterId + grandId + challengerId))
print(len(unq_game_upper)) # 57만 1149개

571149


In [5]:
save_pickle(unq_game_upper, "unique_matchId_upper")

### master match 파일에서 문제 찾기
왜 np.where으로 인덱싱한 결과에선 0 이 '0'으로 나올까? --> __python list를 numpy array로 변환할 때 dtype 통일됨 주의__
```python
clist = np.array([0, 1, 2, 'string', 'string2'])
```
로 하면 `clist`는 `np.array(['0', '1', '2', 'string', 'string2'], dtype='<U21')`가 된다. 즉, int와 string이 섞여있는 key의 list를 np.array로 변환하면서 모두 string으로 변환됨. 근데 np.array가 아니면 리스트로 인덱싱을 하는게 불가능하다.
1. numpy array의 element를 object로 설정
```python
alist = np.array([0, 1, 2, 'string', 'string2'], dtype='O')
>> array([0, 1, 2, 'string', 'string2'], dtype=object)
```

2. 리스트로 리스트 인덱싱하기
```python
T = [List[i] for i in Idx]
```

In [90]:
keys = np.array(list(master.keys()))
values = np.array(list(master.values()))

test = np.array([str in [type(match) for match in matchList] for matchList in values])

In [91]:
keys[np.where(test == True)]

array(['0', '1', '2', '3', '4', '5', '6', '7', '8'], dtype='<U56')

In [54]:
values[np.where(test == True)]

<class 'numpy.ndarray'>


array(['IDFbOhL9zrM92kxqQ_VP8xY-l_wPPclamwrSXTq_sRhxXyY',
       'dmN8cc-Rh-LxTfbHsAWlTFFgONC36HSkVT5ADdGUlTuPaK4',
       'PfbIFOEHNiIKI_Kdlm01s8VADcjRnJIMj0Az4BGMlL2YhZNp_ni8-zbq',
       'tEXMnf91BBc8huUQC4gNW8qbg5WRThYE3F_2O2xMCVkhnheFGdS5wHH3',
       'QWCR8H6UHQM5j4v6ZtPSn0Eei1p35VKK7h2f3ri5UMpir5rcYxGG3pKZ',
       'VbXsuxQ5UIBALVTIpXRIrBRQwdtNZVgzqxCvMSyemOMc',
       'j1_Xvl5d9xgSDoqepuEfGP3VQr9VLzVJw1JVD9YCjhzEnhQ',
       '3l385LpRZawrBPeVjVbyBCCpKn35CE5dWZLUnv5GHjro6Og',
       'FpclnRa3_e3ItlIbkTLVy433gtAzn-ZDwIeedgQRSWeqO_I'], dtype=object)

In [64]:
errorkey = keys[np.where(test == True)] # '0'은 없고 0은 있다...
errorkey

array(['0', '1', '2', '3', '4', '5', '6', '7', '8'], dtype='<U56')

In [95]:
for key in errorkey:
    key = int(key)
    if key in master.keys(): 
        del master[key]
        print('{} deleted in master'.format(key))

0 deleted in master
1 deleted in master
2 deleted in master
3 deleted in master
4 deleted in master
5 deleted in master
6 deleted in master
7 deleted in master
8 deleted in master


In [97]:
save_pickle(master, 'match_MASTER')

# 사전준비   
랭크별 summoner name의 list   
https://developer.riotgames.com/api-methods/#league-v4/GET_getLeagueEntries
- tiers = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']
- divisions = ['I', 'II', 'III', 'IV']
- queue = ['RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_FLEX_TT']  

랭크별 account의 list: { 랭크이름: account_id의 list } 

In [ ]:
tier = ['MASTER', 'GRANDMASTER', 'CHALLENGER']
divisions = ['I', 'II', 'III', 'IV']
queue = 'RANKED_SOLO_5x5' # 솔랭을 기준으로


# for t in divisions:
#     summoners = []
#     summoners.extend(get_entry(queue, t, div))

In [ ]:
# MASTER
json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{0}?api_key={1}".format(queue, api_key))
summoner = [i['summonerId'] for i in json_data['entries']]

with open('pickle/MASTER.pickle', 'wb') as f:
    pickle.dump(summoner, f)

# GRANDMASTER
json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/{0}?api_key={1}".format(queue, api_key))
summoner = [i['summonerId'] for i in json_data['entries']]

with open('pickle/GRANDMASTER.pickle', 'wb') as f:
    pickle.dump(summoner, f)

# CHALLENGER
json_data = RiotRequest("https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{0}?api_key={1}".format(queue, api_key))
summoner = [i['summonerId'] for i in json_data['entries']]

with open('pickle/CHALLENGER.pickle', 'wb') as f:
    pickle.dump(summoner, f)

In [ ]:
len(summoner)

# Account ID
5월 28일 화요일 새벽 1:28 MASTER와 CHALLENGER에서는 에러 안났고 GRANDMASTER에서 error 두 개 났는데 다시 해봐도 안나오길래 우선 무시하고 진행

In [ ]:
import datetime

for t in tier:
    with open('pickle/{}.pickle'.format(t), 'rb') as f:
        summoners = pickle.load(f)
    
    # get a list of account id for a list of summoner id
    accounts = []
    error = []

    
    for index, smr in enumerate(summoners): 

        try:
            accounts.append(get_account_id_by_summonerId(smr))
            print("Summoner No.{} Collected".format(str(index)))
        except:
            error.append(smr)

        if index%1000 == 0:
            # save as pickle. 100개 마다 덮어쓰기
            fn = "pickle/account_{}.pickle".format(t)
            with open(fn, 'wb') as f:
                pickle.dump(accounts, f)

            fn = "pickle/account_error_{}.pickle".format(t)
            with open(fn, 'wb') as f:
                pickle.dump(error, f)

            clear_output()
            print("================== Colleting Summoner Id from NO.{0} of {1} ====================".format(str(index+1), t))
            now = datetime.datetime.now()
            print("==================" + now.strftime("%m월 %d일 %H:%M") + "====================" )
    
    # a loop within a tier is completed
    fn = "pickle/account_{}.pickle".format(t)
    with open(fn, 'wb') as f:
        pickle.dump(accounts, f)

    fn = "pickle/account_error_{}.pickle".format(t)
    with open(fn, 'wb') as f:
        pickle.dump(error, f)

In [ ]:
t = 'GRANDMASTER'
with open('pickle/account_error_{}.pickle'.format(t), 'rb') as f:
    error = pickle.load(f)
error # get_account_id 해봐도 No Match Found --> 우선 넘어간다

# MatchReference History
https://developer.riotgames.com/api-methods/#match-v4/GET_getMatchlist   
: { account_id : MatchReference의 list }  
- CHALLENGER: 2개 400 에러
- MASTER: 9개 모두 추가 완료
- GRANDMASTER: 3개 모두 추가 완료

5월 28일 화요일 미팅 후 찬영이 남은 거 돌리기 약 7016개

In [2]:
###### DIAMOND 3
t = 'DIA3'

with open('pickle/D3_4.pickle'.format(t), 'rb') as f:
    account4 = pickle.load(f)

with open('pickle/D3_5.pickle'.format(t), 'rb') as f:
    account5 = pickle.load(f)

account = account4 + account5
len(account)

7016

In [6]:
with open('compress.pickle'.format(t), 'rb') as f:
    test = pickle.load(f)

In [16]:
summoner_name = "marwin"
account_id = get_account_id("marwin")
account_id

'm15oeLWrxjD0ITBuP7sW8JhLJoD6f-ZiH6HeGsa7HddFxTjwugMK_jk8'

In [20]:
get_this_season_match_history('xQ3KwYZcBXp-bAOm5fU38e-qE5AQIYMFxMYWp2MpKnpH')

[{'platformId': 'KR',
  'gameId': 3681215367,
  'champion': 111,
  'queue': 430,
  'season': 13,
  'timestamp': 1559054623959,
  'role': 'DUO_SUPPORT',
  'lane': 'MID'},
 {'platformId': 'KR',
  'gameId': 3681147944,
  'champion': 119,
  'queue': 430,
  'season': 13,
  'timestamp': 1559052293001,
  'role': 'DUO_CARRY',
  'lane': 'BOTTOM'},
 {'platformId': 'KR',
  'gameId': 3681162467,
  'champion': 101,
  'queue': 430,
  'season': 13,
  'timestamp': 1559050880256,
  'role': 'DUO_SUPPORT',
  'lane': 'NONE'},
 {'platformId': 'KR',
  'gameId': 3681094351,
  'champion': 101,
  'queue': 430,
  'season': 13,
  'timestamp': 1559048582907,
  'role': 'DUO',
  'lane': 'MID'},
 {'platformId': 'KR',
  'gameId': 3681065701,
  'champion': 350,
  'queue': 430,
  'season': 13,
  'timestamp': 1559046368788,
  'role': 'DUO_SUPPORT',
  'lane': 'BOTTOM'},
 {'platformId': 'KR',
  'gameId': 3681010602,
  'champion': 555,
  'queue': 430,
  'season': 13,
  'timestamp': 1559044553451,
  'role': 'DUO_CARRY',
  '

In [18]:
get_game_detail(3681147944)

{'gameId': 3681147944,
 'platformId': 'KR',
 'gameCreation': 1559052293001,
 'gameDuration': 2121,
 'queueId': 430,
 'mapId': 11,
 'seasonId': 13,
 'gameVersion': '9.10.274.8345',
 'gameMode': 'CLASSIC',
 'gameType': 'MATCHED_GAME',
 'teams': [{'teamId': 100,
   'win': 'Fail',
   'firstBlood': False,
   'firstTower': False,
   'firstInhibitor': False,
   'firstBaron': False,
   'firstDragon': False,
   'firstRiftHerald': False,
   'towerKills': 4,
   'inhibitorKills': 0,
   'baronKills': 0,
   'dragonKills': 0,
   'vilemawKills': 0,
   'riftHeraldKills': 0,
   'dominionVictoryScore': 0,
   'bans': []},
  {'teamId': 200,
   'win': 'Win',
   'firstBlood': True,
   'firstTower': True,
   'firstInhibitor': True,
   'firstBaron': True,
   'firstDragon': True,
   'firstRiftHerald': False,
   'towerKills': 10,
   'inhibitorKills': 2,
   'baronKills': 1,
   'dragonKills': 4,
   'vilemawKills': 0,
   'riftHeraldKills': 0,
   'dominionVictoryScore': 0,
   'bans': []}],
 'participants': [{'partic

In [19]:
get_this_season_match_history('5YSbGtHPmrwaFBjEVOVzvMJJhLcb0nPeU1v6dB47kG-y')

Unhandled Error 400 : https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/5YSbGtHPmrwaFBjEVOVzvMJJhLcb0nPeU1v6dB47kG-y?api_key=RGAPI-5d4d9788-7f08-485b-86de-c8528dc9baac&beginIndex=0


TypeError: 'NoneType' object is not subscriptable

In [ ]:
###### MASTER / GRANDMASTER / CHALLENGER
# for t in tier:
#     # open a list of account id in the tier
#     with open('pickle/account_{}.pickle'.format(t), 'rb') as f:
#         accounts = pickle.load(f)

hst_error =[] # list entity to save account ids of incomplete requests
match_history = {} # dictionary entity to save the result of completed requests

for index, acc in enumerate(account):
    try:
        match_history[acc] = get_this_season_match_history(acc)
        print("Match History for Account No.{} Collected".format(str(index)))
    except:
        hst_error.append(acc)
        print("Failed: Collecting Match History for Account No.{}".format(str(index)))

    if index%100 == 0:
        # save as pickle. 100개 마다 덮어쓰기
        fn = "pickle/match_{}.pickle".format(t)
        with open(fn, 'wb') as f:
            pickle.dump(match_history, f)

        fn = "pickle/match_error_{}.pickle".format(t)
        with open(fn, 'wb') as f:
            pickle.dump(hst_error, f)

        clear_output()
        print("================== Colleting Match history from Account No.{} ====================".format(str(index+1)))
        now = datetime.datetime.now()
        print("==================" + now.strftime("%m월 %d일 %H:%M") + "====================" )

# for loop within the tier is completed
fn = "pickle/match_{}.pickle".format(t)
with open(fn, 'wb') as f:
    pickle.dump(match_history, f)

fn = "pickle/match_error_{}.pickle".format(t)
with open(fn, 'wb') as f:
    pickle.dump(hst_error, f)

        

================== Colleting Match history from Account No.1 ====================
==================05월 28일 23:28====================
Unhandled Error 400 : https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/VAXp4odgc5PDujU8-RGtbrzI0EYCbg3XxZj6SikKNwuLHmU?api_key=RGAPI-5d4d9788-7f08-485b-86de-c8528dc9baac&beginIndex=0
Failed: Collecting Match History for Account No.1
Unhandled Error 400 : https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/MajA43daML7bbwBORo5wwlkUT-vQ2Y9KVrkCyBmmB5nQtjs?api_key=RGAPI-5d4d9788-7f08-485b-86de-c8528dc9baac&beginIndex=0
Failed: Collecting Match History for Account No.2
Unhandled Error 400 : https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/Nh3J63bdawz4DQfpstX5eC6P_Udngr9pxCQUtFW1DRL6?api_key=RGAPI-5d4d9788-7f08-485b-86de-c8528dc9baac&beginIndex=0
Failed: Collecting Match History for Account No.3
Unhandled Error 400 : https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/iB_1XiXXMVH5NHuCmsjRzHJEHVAW2aKslSvIGuz

## Length Exploring

In [ ]:
tier = ['MASTER', 'GRANDMASTER', 'CHALLENGER']

with open('pickle/match_{}.pickle'.format(tier[0]), 'rb') as f:
    match = pickle.load(f)

masterLength = [len(value) for key, value in match.items()]

In [ ]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
%matplotlib inline

num_bins = 10
n, bins, patches = plt.hist(masterLength, num_bins, facecolor='blue', alpha=0.5)
plt.show()

In [ ]:
with open('pickle/match_{}.pickle'.format(tier[1]), 'rb') as f:
    match = pickle.load(f)

grandMasterLength = [len(value) for key, value in match.items()]

num_bins = 10
n, bins, patches = plt.hist(grandMasterLength, num_bins, facecolor='blue', alpha=0.5)
plt.show()

In [ ]:
with open('pickle/match_{}.pickle'.format(tier[2]), 'rb') as f:
    match = pickle.load(f)

challengerLength = [len(value) for key, value in match.items()]

num_bins = 10
n, bins, patches = plt.hist(challengerLength, num_bins, facecolor='blue', alpha=0.5)
plt.show()

In [ ]:
print(np.average(masterLength), np.average(grandMasterLength), np.average(challengerLength))

# match_id 의 MatchDto  
https://developer.riotgames.com/api-methods/#match-v4/GET_getMatch   
: {match_id: Match}

In [ ]:
with open('pickle/tmp_bronze_match_history.pickle', 'rb') as f:
    match_history = pickle.load(f)

# list up unique game id
games = [match['gameId'] for key, matchList in match_history.items() for match in matchList]
unq_games = list(set(games))

print(len(unq_games))
print(unq_games[:10])

In [ ]:
fn = "pickle/unq_games.pickle"
with open(fn, 'rb') as f:
    unq_games = pickle.load(f)

In [ ]:
for i in range(0, 6984):
    
    # save as file for every 100 game detail
    rng = (i*100, (i+1)*100)

    match_detail = {} # dictionary to save game detail
    detail_error = [] # list of broken match id

    # starting requests
    start = time.time() 
    
    for index, match in unq_games[rng[0]:rng[1]]:
        try:
            match_detail[match] = get_match_detail(match)
        except:
            print("Match No.{} Broken".format(str(index)))
            detail_error.append(match)
    
    # requests completed
    end = time.time() 
    delta = (end - start)/60
    print("======= {0}th 100 Matches Completed: took {1} minutes =====".format(str(i), str(delta))
    
    # save as pickle
    fn = "pickle/detail/match_detail"+str(rng[0])+"_"+str(rng[1])+".pickle"
    with open(fn, 'wb') as f:
        pickle.dump(match_detail, f)

    fn = "pickle/detail/detail_error"+str(rng[0])+"_"+str(rng[1])+".pickle"
    with open(fn, 'wb') as f:
        pickle.dump(detail_error, f)